Linajea Tracking Example
=====================


This example show all steps necessary to generate the final tracks, from training the network to finding the optimal ILP weights on the validation data to computing the tracks on the test data.

- train network
- predict on validation data
- grid search weights for ILP
  - solve once per set of weights
  - evaluate once per set of weights
  - select set with fewest errors
- predict on test data
- solve on test data with optimal weights
- evaluate on test data

In [1]:
%load_ext autoreload
%autoreload 2
import logging
import os
import sys
import time
import types

import numpy as np
import pandas as pd

from linajea.config import TrackingConfig
import linajea.evaluation
from linajea.process_blockwise import (extract_edges_blockwise,
                                       predict_blockwise,
                                       solve_blockwise)
from linajea.training import train
import linajea.config
import linajea.process_blockwise
import linajea

In [2]:
logging.basicConfig(
        level=logging.DEBUG,
        format='%(asctime)s %(name)s %(levelname)-8s %(message)s')

Configuration
--------------------

All parameters to control the pipeline (e.g. model architecture, data augmentation, training parameters, ILP weights) are contained in a configuration file (in the TOML format https://toml.io)

You can use a single monolithic configuration file or separate configuration files for a subset of the steps of the pipeline, as long as the parameters required for the respective steps are there.

Familiarize yourself with the example configuration files and have a look at the documentation for the configuration to see what is needed. Most parameters have sensible defaults; usually setting the correct paths and the data configuration is all that is needed to start. See `run_multiple_samples.ipynb` for an example setup that can (optionally) handle multiple samples and automates the process of selecting the correct data for each step as much as possible. For training `train_data` has to be set, and for validation and testing `inference_data`.

You can modify the `config_file` variable to point to the config file you would like to use. Make sure that the file paths contained in it point to the correct destination, for instance that they are adapted to your directory structure.

In [3]:
train_config_file = "config_example_single_sample_train.toml"
train_config = TrackingConfig.from_file(train_config_file)
os.makedirs(train_config.general.setup_dir, exist_ok=True)

Training
------------

To start training simply pass the configuration object to the train function. Make sure that the training data and parameters such as the number of iterations/setps are set correctly.

To train until convergence will take from several hours to multiple days.

In [ ]:
train(train_config)

Validation
--------------

After the training is completed we first have to determine the optimal ILP weights.
This is achieved by first creating the prediction on the validation data and then performing a grid search by solving the ILP and evaluating the results repeatedly.

MongoDB is used to store the computed results. A `mongod` server has to be running before executing the remaining cells.
See https://www.mongodb.com/docs/manual/administration/install-community/ for a guide on how to install it (Linux/Windows/MacOS)
Alternatively you might want to create a singularity image (https://github.com/singularityhub/mongo). This can be used locally but will be necessary if you want to run the code on an HPC cluster and there is no server installed already.

In [3]:
validation_config_file = "config_example_single_sample_val.toml"
val_config = TrackingConfig.from_file(validation_config_file)
os.makedirs(val_config.general.setup_dir, exist_ok=True)

### Predict Validation Data

To predict the `cell_indicator` and `movement_vectors` on the validation data make sure that `args.validation` is set to `True`, then execute the next cell.

Depending on the number of workers used (see config file) and the size of the data this can take a while.

In [ ]:
predict_blockwise(val_config)

### Extract Edges Validation Data

For each detected cell, look for neighboring cells in the next time frame and insert an edge candidate for each into the database.

In [ ]:
extract_edges_blockwise(val_config)

### ILP Weights Grid Search

#### Solve on Validation Data

Make sure that `solve.grid_search` is set to `True`. The parameter sets to try are generated automatically.

In [ ]:
linajea.process_blockwise.solve_blockwise(val_config)

#### Evaluate on Validation Data

In [5]:
validation_config_file = "config_example_single_sample_val.toml"
val_config = TrackingConfig.from_file(validation_config_file)
parameters = val_config.solve.parameters
for params in parameters:
    val_config.solve.parameters = [params]
    linajea.evaluation.evaluate_setup(val_config)

2022-07-05 08:12:02,499 linajea.utils.candidate_database INFO     Querying ID for parameters {'track_cost': 7, 'weight_node_score': -17, 'selection_constant': 12, 'weight_division': -11, 'division_constant': 2.5, 'weight_child': 2.0, 'weight_continuation': -1.0, 'weight_edge_score': 0.35, 'block_size': [15, 512, 512, 712], 'context': [2, 100, 100, 100], 'feature_func': 'noop', 'val': True, 'solver_type': 'basic', 'max_cell_move': {'$exists': False}, 'tag': {'$exists': False}}


AssertionError: Did not find id for parameters {'track_cost': 7, 'weight_node_score': -17, 'selection_constant': 12, 'weight_division': -11, 'division_constant': 2.5, 'weight_child': 2.0, 'weight_continuation': -1.0, 'weight_edge_score': 0.35, 'block_size': [15, 512, 512, 712], 'context': [2, 100, 100, 100], 'feature_func': 'noop', 'val': True, 'solver_type': 'basic', 'max_cell_move': {'$exists': False}, 'tag': {'$exists': False}} in linajea_celegans_20220624_134111b and fail_if_not_exists set to True

### Predict Test Data

Now that we know which ILP weights to use we can predict the `cell_indicator` and `movement_vectors` on the test data and compute the tracks. Make sure that `args.validation` is set to `False` and `solve.grid_search` and `solve.random_search` are set to `False`.

In [4]:
test_config_file = "config_example_single_sample_test.toml"
test_config = TrackingConfig.from_file(test_config_file)
print(test_config.inference_data.data_source)

DataSourceConfig(datafile=DataFileConfig(filename='/nrs/funke/hirschp/mskcc_emb3', group='volumes/raw_nested', file_roi=DataROIConfig(offset=[0, 0, 0, 0], shape=[425, 205, 512, 512]), file_voxel_size=[1, 5, 1, 1]), tracksfile=None, divisionsfile=None, daughtersfile=None, db_name='linajea_celegans_20220623_180937', gt_db_name='linajea_celegans_emb3_fixed', gt_db_name_polar=None, voxel_size=[1, 5, 1, 1], roi=DataROIConfig(offset=[50, 0, 0, 0], shape=[10, 205, 512, 512]), exclude_times=None)


In [ ]:
predict_blockwise(test_config)

### Solve on Test Data

Then we can solve the ILP on the test data. We select the ILP weights that resulted in the lowest overall number of errors on the validation data.

In [5]:
score_columns = ['fn_edges', 'identity_switches',
                 'fp_divisions', 'fn_divisions']
if not val_config.general.sparse:
    score_columns = ['fp_edges'] + score_columns

results = linajea.evaluation.get_results_sorted(
    val_config,
    filter_params={"val": True},
    score_columns=score_columns,
    sort_by="sum_errors")

test_config.solve.parameters = [val_config.solve.parameters[0]]
test_config.solve.parameters[0].weight_node_score = float(results.iloc[0].weight_node_score)
test_config.solve.parameters[0].selection_constant = float(results.iloc[0].selection_constant)
test_config.solve.parameters[0].track_cost = float(results.iloc[0].track_cost)
test_config.solve.parameters[0].weight_edge_score = float(results.iloc[0].weight_edge_score)
test_config.solve.parameters[0].weight_division = float(results.iloc[0].weight_division)
test_config.solve.parameters[0].weight_child = float(results.iloc[0].weight_child)
test_config.solve.parameters[0].weight_continuation = float(results.iloc[0].weight_continuation)
print(test_config.solve.parameters[0], type(test_config.solve.parameters[0].weight_continuation))

2022-07-05 08:39:50,763 linajea.evaluation.analyze_results INFO     Getting results in db: linajea_celegans_20220624_134111b
2022-07-05 08:39:50,783 linajea.utils.candidate_database INFO     Query: {'val': True, 'matching_threshold': 15, 'validation_score': False, 'window_size': 50, 'ignore_one_off_div_errors': False, 'fn_div_count_unconnected_parent': True}
2022-07-05 08:39:50,785 linajea.utils.candidate_database INFO     Found 5 scores


SolveParametersConfig(track_cost=7.0, weight_node_score=-17.0, selection_constant=12.0, weight_division=-11.0, division_constant=2.5, weight_child=2.0, weight_continuation=-1.0, weight_edge_score=0.35, block_size=[15, 512, 512, 712], context=[2, 100, 100, 100], max_cell_move=None, roi=None, feature_func='noop', val=True, tag=None, solver_type='basic') <class 'float'>


In [6]:
print(test_config.solve.parameters)
solve_blockwise(test_config)

2022-07-05 08:39:53,619 linajea.utils.candidate_database INFO     Querying ID for parameters {'track_cost': 7.0, 'weight_node_score': -17.0, 'selection_constant': 12.0, 'weight_division': -11.0, 'division_constant': 2.5, 'weight_child': 2.0, 'weight_continuation': -1.0, 'weight_edge_score': 0.35, 'block_size': [15, 512, 512, 712], 'context': [2, 100, 100, 100], 'feature_func': 'noop', 'val': True, 'solver_type': 'basic', 'max_cell_move': {'$exists': False}, 'tag': {'$exists': False}}
2022-07-05 08:39:53,621 linajea.utils.candidate_database INFO     Parameters {'track_cost': 7.0, 'weight_node_score': -17.0, 'selection_constant': 12.0, 'weight_division': -11.0, 'division_constant': 2.5, 'weight_child': 2.0, 'weight_continuation': -1.0, 'weight_edge_score': 0.35, 'block_size': [15, 512, 512, 712], 'context': [2, 100, 100, 100], 'feature_func': 'noop', 'val': True, 'solver_type': 'basic', 'max_cell_move': {'$exists': False}, 'tag': {'$exists': False}} already in collection with id 7
2022-0

[SolveParametersConfig(track_cost=7.0, weight_node_score=-17.0, selection_constant=12.0, weight_division=-11.0, division_constant=2.5, weight_child=2.0, weight_continuation=-1.0, weight_edge_score=0.35, block_size=[15, 512, 512, 712], context=[2, 100, 100, 100], max_cell_move=None, roi=None, feature_func='noop', val=True, tag=None, solver_type='basic')]


2022-07-05 08:39:53,683 daisy.client DEBUG    Client init
2022-07-05 08:39:53,686 daisy.client DEBUG    Client context: hostname=10.36.110.14:port=22467:task_id=linajea_solving:worker_id=0
2022-07-05 08:39:53,688 daisy.tcp.io_looper DEBUG    Creating new IOLoop for process 3276962...
2022-07-05 08:39:53,689 asyncio DEBUG    Using selector: EpollSelector
2022-07-05 08:39:53,690 daisy.tcp.io_looper DEBUG    Starting io loop for process 3276962...
2022-07-05 08:39:53,690 daisy.tcp.tcp_client DEBUG    Creating new TCP client...
2022-07-05 08:39:53,691 daisy.tcp.tcp_client DEBUG    Connecting to server at 10.36.110.14:22467...
2022-07-05 08:39:53,693 daisy.tcp.io_looper DEBUG    Reusing IOLoop for process 3276962...
2022-07-05 08:39:53,693 daisy.tcp.tcp_server DEBUG    Received new connection from 10.36.110.14:58822
2022-07-05 08:39:53,694 daisy.tcp.io_looper DEBUG    Reusing IOLoop for process 3276896...
2022-07-05 08:39:53,792 daisy.tcp.tcp_client DEBUG    ...connected
2022-07-05 08:39:53

linajea_solving ▶:   0%|          | 0/1 [00:00<?, ?blocks/s]

2022-07-05 08:39:53,831 daisy.server DEBUG    Task state for task linajea_solving: Started: True
Total Blocks: 1
Ready: 0
Processing: 1
Pending: 0
Completed: 0
Skipped: 0
Failed: 0
Orphaned: 0

2022-07-05 08:39:53,828 daisy.client DEBUG    Received block ('linajea_solving', 1)
2022-07-05 08:39:53,832 daisy.server DEBUG    Task linajea_solving has 0 ready blocks
2022-07-05 08:39:53,829 linajea.process_blockwise.solve_blockwise INFO     Solving in block linajea_solving/1 with read ROI [48:67, -100:612, -100:612, -100:812] (19, 712, 712, 912) and write ROI [50:65, 0:512, 0:512, 0:712] (15, 512, 512, 712)
2022-07-05 08:39:53,830 linajea.process_blockwise.solve_blockwise DEBUG    Block write roi: [50:65, 0:512, 0:512, 0:712] (15, 512, 512, 712)
2022-07-05 08:39:53,831 linajea.process_blockwise.solve_blockwise DEBUG    Solution roi: [50:60, 0:205, 0:512, 0:512] (10, 205, 512, 512)
2022-07-05 08:39:53,831 linajea.process_blockwise.solve_blockwise DEBUG    Write roi: [50:60, 0:205, 0:512, 0:51

...

2022-07-05 08:39:54,362 daisy.persistence.mongodb_graph_provider DEBUG    Checking if coordinate [54, 150, 211, 325] is inside roi [50:60, 0:205, 0:512, 0:512] (10, 205, 512, 512)
2022-07-05 08:39:54,362 daisy.persistence.mongodb_graph_provider DEBUG    Checking if coordinate [54, 150, 318, 226] is inside roi [50:60, 0:205, 0:512, 0:512] (10, 205, 512, 512)
2022-07-05 08:39:54,363 daisy.persistence.mongodb_graph_provider DEBUG    Checking if coordinate [54, 155, 261, 272] is inside roi [50:60, 0:205, 0:512, 0:512] (10, 205, 512, 512)
2022-07-05 08:39:54,363 daisy.persistence.mongodb_graph_provider DEBUG    Checking if coordinate [54, 155, 278, 339] is inside roi [50:60, 0:205, 0:512, 0:512] (10, 205, 512, 512)
2022-07-05 08:39:54,364 daisy.persistence.mongodb_graph_provider DEBUG    Checking if coordinate [54, 165, 219, 289] is inside roi [50:60, 0:205, 0:512, 0:512] (10, 205, 512, 512)
2022-07-05 08:39:54,364 daisy.persistence.mongodb_graph_provider DEBUG    Checking if coordinate [54

2022-07-05 08:39:54,380 daisy.persistence.mongodb_graph_provider DEBUG    Checking if coordinate [55, 165, 205, 290] is inside roi [50:60, 0:205, 0:512, 0:512] (10, 205, 512, 512)
2022-07-05 08:39:54,381 daisy.persistence.mongodb_graph_provider DEBUG    Checking if coordinate [55, 165, 205, 290] is inside roi [50:60, 0:205, 0:512, 0:512] (10, 205, 512, 512)
2022-07-05 08:39:54,381 daisy.persistence.mongodb_graph_provider DEBUG    Checking if coordinate [55, 165, 205, 290] is inside roi [50:60, 0:205, 0:512, 0:512] (10, 205, 512, 512)
2022-07-05 08:39:54,382 daisy.persistence.mongodb_graph_provider DEBUG    Checking if coordinate [55, 165, 205, 290] is inside roi [50:60, 0:205, 0:512, 0:512] (10, 205, 512, 512)
2022-07-05 08:39:54,382 daisy.persistence.mongodb_graph_provider DEBUG    Checking if coordinate [55, 175, 177, 301] is inside roi [50:60, 0:205, 0:512, 0:512] (10, 205, 512, 512)
2022-07-05 08:39:54,382 daisy.persistence.mongodb_graph_provider DEBUG    Checking if coordinate [55

2022-07-05 08:39:54,399 daisy.persistence.mongodb_graph_provider DEBUG    Checking if coordinate [56, 165, 205, 290] is inside roi [50:60, 0:205, 0:512, 0:512] (10, 205, 512, 512)
2022-07-05 08:39:54,399 daisy.persistence.mongodb_graph_provider DEBUG    Checking if coordinate [56, 165, 205, 290] is inside roi [50:60, 0:205, 0:512, 0:512] (10, 205, 512, 512)
2022-07-05 08:39:54,400 daisy.persistence.mongodb_graph_provider DEBUG    Checking if coordinate [56, 165, 205, 290] is inside roi [50:60, 0:205, 0:512, 0:512] (10, 205, 512, 512)
2022-07-05 08:39:54,400 daisy.persistence.mongodb_graph_provider DEBUG    Checking if coordinate [56, 165, 205, 290] is inside roi [50:60, 0:205, 0:512, 0:512] (10, 205, 512, 512)
2022-07-05 08:39:54,401 daisy.persistence.mongodb_graph_provider DEBUG    Checking if coordinate [56, 165, 205, 290] is inside roi [50:60, 0:205, 0:512, 0:512] (10, 205, 512, 512)
2022-07-05 08:39:54,401 daisy.persistence.mongodb_graph_provider DEBUG    Checking if coordinate [56


Execution Summary
-----------------

  Task linajea_solving:

    num blocks : 1
    completed ✔: 1 (skipped 0)
    failed    ✗: 0
    orphaned  ∅: 0

    all blocks processed successfully


[7]

### Evaluate on Test Data

In [ ]:
report = linajea.evaluation.evaluate_setup(test_config)
print(report.get_short_report())